In [ ]:
import hashlib
import hmac
import time
import base64
from urllib.parse import quote_plus
import requests
import pandas as pd
import logging
from datetime import datetime

# Configurar el logger
logging.basicConfig(filename='whalewisdom_api.log', level=logging.INFO,
                    format='%(asctime)s:%(levelname)s:%(message)s')

# Definir las claves de acceso API
shared_key = 'wY0e3zria06ULMWuSmR2'
secret_key = 'MJ4RWDjzZxtA36KBZh5LMlDKMrw9m0cHpfRzrgH0'

# Leer el DataFrame de filer_ids desde un archivo CSV
filers = pd.read_csv('filers.csv')

# Renombrar la columna 'id' a 'filer_id'
filers = filers.rename(columns={'id': 'filer_id'})

# Lista para almacenar los holdings
holdings_list = []

# Lista para almacenar los tiempos de extracción
extraction_times = []

def fetch_holdings(filer_id):
    logging.info(f"Procesando filer_id: {filer_id}")
    
    json_args = f'{{"command":"holdings","filer_ids":[{filer_id}],"quarter_ids":[96,97],"all_quarters":0,"include_13d":1}}'
    formatted_args = quote_plus(json_args)
    timenow = time.strftime('%Y-%m-%dT%H:%M:%SZ', time.gmtime())

    digest = hashlib.sha1
    raw_args = json_args + '\n' + timenow
    hmac_hash = hmac.new(secret_key.encode(), raw_args.encode(), digest).digest()
    sig = base64.b64encode(hmac_hash).rstrip()

    url_base = 'https://whalewisdom.com/shell/command.json?'
    url_args = 'args=' + formatted_args
    url_end = f'&api_shared_key={shared_key}&api_sig={sig.decode()}&timestamp={timenow}'
    api_url = url_base + url_args + url_end

    try:
        logging.info(f"Realizando solicitud a la API para filer_id: {filer_id}")
        response = requests.get(api_url)
        response.raise_for_status()  # Lanza una excepción para códigos de estado HTTP 4xx/5xx
        data = response.json()
        results = data.get('results', [])

        for result in results:
            filer_name = result['filer_name']
            for record in result['records']:
                for holding in record['holdings']:
                    holding['filer_name'] = filer_name
                    holdings_list.append(holding)
        logging.info(f"Datos obtenidos para filer_id: {filer_id}")
        logging.info(f"Consulta exitosa para filer_id: {filer_id}")

    except requests.exceptions.RequestException as e:
        logging.error(f"Error al conectar con la API de Whale Wisdom para filer_id {filer_id}: {e}")
        print(f"Error al conectar con la API de Whale Wisdom para filer_id {filer_id}: {e}")

# Iterar sobre todos los registros en filers de forma secuencial
start_time = datetime.now()

#for index, row in filers.iterrows():
fetch_holdings(71)

end_time = datetime.now()
extraction_time = (end_time - start_time).total_seconds()
extraction_times.append({'total_records': len(filers), 'extraction_time': extraction_time})
logging.info(f"Tiempo total de extracción para {len(filers)} filer_ids: {extraction_time} segundos")

# Almacenar la información en un DataFrame
holdings_df = pd.DataFrame(holdings_list)
logging.info("Datos almacenados en el DataFrame")
print(holdings_df)

# Crear un DataFrame con los tiempos de extracción
extraction_times_df = pd.DataFrame(extraction_times)
print(extraction_times_df)

In [ ]:
import logging
import pandas as pd
import requests
from urllib.parse import quote_plus
import hmac
import hashlib
import base64
from datetime import datetime

# Configurar el logger
logging.basicConfig(filename='whalewisdom_api.log', level=logging.INFO,
                    format='%(asctime)s:%(levelname)s:%(message)s')

# Definir las claves de acceso API
shared_key = 'wY0e3zria06ULMWuSmR2'
secret_key = 'MJ4RWDjzZxtA36KBZh5LMlDKMrw9m0cHpfRzrgH0'

# Función para generar la URL de la API
def generate_api_url(command, shared_key, secret_key):
    json_args = f'{{"command":"{command}"}}'
    formatted_args = quote_plus(json_args)
    timenow = datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')
    raw_args = f'{json_args}\n{timenow}'
    hmac_hash = hmac.new(secret_key.encode(), raw_args.encode(), hashlib.sha1).digest()
    sig = base64.b64encode(hmac_hash).decode().rstrip()
    return (f'https://whalewisdom.com/shell/command.json?args={formatted_args}'
            f'&api_shared_key={shared_key}&api_sig={sig}&timestamp={timenow}')

# Función para realizar la solicitud a la API
def fetch_data_from_api(api_url):
    try:
        logging.info(f"Realizando solicitud a la API: {api_url}")
        response = requests.get(api_url)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        logging.error(f"Error al conectar con la API: {e}")
        print(f"Error al conectar con la API: {e}")
        return None

# Procesar los datos de los quarters
def process_quarters(data):
    if not data or 'quarters' not in data:
        logging.warning("No se encontraron datos de quarters en la respuesta.")
        return []

    quarters = data['quarters']
    logging.info(f"Se obtuvieron {len(quarters)} quarters.")
    return quarters

# Medir el tiempo de extracción
start_time = datetime.now()

# Generar la URL y obtener los datos
api_url = generate_api_url("quarters", shared_key, secret_key)
data = fetch_data_from_api(api_url)

# Procesar los datos obtenidos
quarter_list = process_quarters(data)

# Calcular el tiempo de extracción
end_time = datetime.now()
extraction_time = (end_time - start_time).total_seconds()
logging.info(f"Tiempo total de extracción: {extraction_time} segundos")

# Crear un DataFrame con los quarters
quarters_df = pd.DataFrame(quarter_list)
print(quarters_df)


In [ ]:
import logging
import pandas as pd
import requests
from urllib.parse import quote_plus
import hmac
import hashlib
import base64
from datetime import datetime
import os

# Configurar el logger
logging.basicConfig(filename='whalewisdom_api.log', level=logging.INFO,
                    format='%(asctime)s:%(levelname)s:%(message)s')

# Definir las claves de acceso API
shared_key = 'wY0e3zria06ULMWuSmR2'
secret_key = 'MJ4RWDjzZxtA36KBZh5LMlDKMrw9m0cHpfRzrgH0'

# Archivo para almacenar los quarters previamente procesados
quarters_file = 'quarters.csv'

# Función para generar la URL de la API
def generate_api_url(command, shared_key, secret_key, filer_id=None):
    if command == "quarters":
        json_args = f'{{"command":"{command}"}}'
    elif command == "holdings" and filer_id:
        json_args = f'{{"command":"holdings","filer_ids":[{filer_id}],"include_13d":1}}'
    else:
        raise ValueError("Comando o argumentos inválidos para generar la URL.")
    
    formatted_args = quote_plus(json_args)
    timenow = datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')
    raw_args = f'{json_args}\n{timenow}'
    hmac_hash = hmac.new(secret_key.encode(), raw_args.encode(), hashlib.sha1).digest()
    sig = base64.b64encode(hmac_hash).decode().rstrip()
    return (f'https://whalewisdom.com/shell/command.json?args={formatted_args}'
            f'&api_shared_key={shared_key}&api_sig={sig}&timestamp={timenow}')

# Función para realizar la solicitud a la API
def fetch_data_from_api(api_url):
    try:
        logging.info(f"Realizando solicitud a la API: {api_url}")
        response = requests.get(api_url)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        logging.error(f"Error al conectar con la API: {e}")
        return None

# Función para procesar los datos de quarters
def process_quarters(data):
    if not data or 'quarters' not in data:
        logging.warning("No se encontraron datos de quarters en la respuesta.")
        return []
    quarters = data['quarters']
    logging.info(f"Se obtuvieron {len(quarters)} quarters.")
    return quarters

# Función para verificar si hay nuevos quarters
def has_new_quarters(current_quarters):
    if os.path.exists(quarters_file):
        previous_quarters = pd.read_csv(quarters_file)['quarters'].tolist()
        if current_quarters == previous_quarters:
            logging.info("No hay nuevos quarters disponibles. Finalizando la ejecución.")
            return False
    return True

# Función para guardar los quarters actuales
def save_quarters(current_quarters):
    pd.DataFrame({'quarters': current_quarters}).to_csv(quarters_file, index=False)
    logging.info("Quarters actuales guardados en el archivo.")

# Función para procesar holdings de un filer_id
def fetch_holdings(filer_id):
    api_url = generate_api_url("holdings", shared_key, secret_key, filer_id=filer_id)
    data = fetch_data_from_api(api_url)
    if not data:
        return []
    
    holdings = []
    results = data.get('results', [])
    for result in results:
        filer_name = result['filer_name']
        for record in result['records']:
            for holding in record['holdings']:
                holding['filer_name'] = filer_name
                holdings.append(holding)
    logging.info(f"Datos obtenidos para filer_id: {filer_id}")
    return holdings

# Medir el tiempo de extracción
start_time = datetime.now()

# Obtener y procesar los datos de quarters
quarters_api_url = generate_api_url("quarters", shared_key, secret_key)
data = fetch_data_from_api(quarters_api_url)
quarter_list = process_quarters(data)

# Verificar si hay nuevos quarters
if not quarter_list or has_new_quarters(quarter_list):
    print("No hay nueva información de quarters. Finalizando la ejecución.")
else:
    # Guardar los quarters actuales
    save_quarters(quarter_list)

    # Leer el DataFrame de filer_ids desde un archivo CSV
    filers = pd.read_csv('filers.csv')
    filers = filers.rename(columns={'id': 'filer_id'})
    filers = filers.iloc[[9]]  # Solo para pruebas, eliminar esta línea en producción

    # Lista para almacenar los holdings
    holdings_list = []

    # Iterar sobre todos los registros en filers
    for index, row in filers.iterrows():
        filer_id = row['filer_id']
        holdings = fetch_holdings(filer_id)  # Ajusta los quarter_ids según sea necesario
        holdings_list.extend(holdings)

    # Almacenar la información en un DataFrame
    holdings_df = pd.DataFrame(holdings_list)
    logging.info("Datos almacenados en el DataFrame")
    print(holdings_df)

# Calcular el tiempo de extracción
end_time = datetime.now()
extraction_time = (end_time - start_time).total_seconds()
logging.info(f"Tiempo total de extracción: {extraction_time} segundos")

In [ ]:
import logging
import pandas as pd
import requests
from urllib.parse import quote_plus
import hmac
import hashlib
import base64
from datetime import datetime
import os

# Configurar el logger
logging.basicConfig(filename='whalewisdom_api.log', level=logging.INFO,
                    format='%(asctime)s:%(levelname)s:%(message)s')

# Definir las claves de acceso API
shared_key = 'wY0e3zria06ULMWuSmR2'
secret_key = 'MJ4RWDjzZxtA36KBZh5LMlDKMrw9m0cHpfRzrgH0'

# Archivo para almacenar los quarters previamente procesados
quarters_file = 'quarters.csv'

# Función para generar la URL de la API
def generate_api_url(command, shared_key, secret_key, filer_id=None):
    if command == "quarters":
        json_args = f'{{"command":"{command}"}}'
    elif command == "holdings" and filer_id:
        json_args = f'{{"command":"holdings","filer_ids":[{filer_id}],"include_13d":1}}'
    else:
        raise ValueError("Comando o argumentos inválidos para generar la URL.")
    
    formatted_args = quote_plus(json_args)
    timenow = datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')
    raw_args = f'{json_args}\n{timenow}'
    hmac_hash = hmac.new(secret_key.encode(), raw_args.encode(), hashlib.sha1).digest()
    sig = base64.b64encode(hmac_hash).decode().rstrip()
    return (f'https://whalewisdom.com/shell/command.json?args={formatted_args}'
            f'&api_shared_key={shared_key}&api_sig={sig}&timestamp={timenow}')

# Función para realizar la solicitud a la API
def fetch_data_from_api(api_url):
    try:
        logging.info(f"Realizando solicitud a la API: {api_url}")
        response = requests.get(api_url)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        logging.error(f"Error al conectar con la API: {e}")
        return None

# Función para procesar los datos de quarters
def process_quarters(data):
    if not data or 'quarters' not in data:
        logging.warning("No se encontraron datos de quarters en la respuesta.")
        return []
    quarters = data['quarters']
    logging.info(f"Se obtuvieron {len(quarters)} quarters.")
    return quarters

# Función para verificar si hay nuevos quarters
def has_new_quarters(current_quarters):
    if os.path.exists(quarters_file):
        # Leer los quarters previamente almacenados
        try:
            previous_quarters = pd.read_csv(quarters_file)['quarters'].apply(eval).tolist()
        except Exception as e:
            logging.error(f"Error al leer el archivo quarters.csv: {e}")
            return True  # Si no se puede leer el archivo, asumir que hay nuevos quarters

        # Comparar los quarters actuales con los previos
        if set(tuple(sorted(q.items())) for q in current_quarters) == set(tuple(sorted(q.items())) for q in previous_quarters):
            logging.info("No hay nuevos quarters disponibles. Finalizando la ejecución.")
            return False
    return True

# Función para guardar los quarters actuales
def save_quarters(current_quarters):
    try:
        pd.DataFrame({'quarters': [str(q) for q in current_quarters]}).to_csv(quarters_file, index=False)
        logging.info("Quarters actuales guardados en el archivo.")
    except Exception as e:
        logging.error(f"Error al guardar los quarters en el archivo: {e}")

# Función para procesar holdings de un filer_id
def fetch_holdings(filer_id):
    api_url = generate_api_url("holdings", shared_key, secret_key, filer_id=filer_id)
    data = fetch_data_from_api(api_url)
    if not data:
        return []
    
    holdings = []
    results = data.get('results', [])
    for result in results:
        filer_name = result['filer_name']
        for record in result['records']:
            for holding in record['holdings']:
                holding['filer_name'] = filer_name
                holdings.append(holding)
    logging.info(f"Datos obtenidos para filer_id: {filer_id}")
    return holdings

# Medir el tiempo de extracción
start_time = datetime.now()

# Obtener y procesar los datos de quarters
quarters_api_url = generate_api_url("quarters", shared_key, secret_key)
data = fetch_data_from_api(quarters_api_url)
quarter_list = process_quarters(data)

# Verificar si hay nuevos quarters
if not quarter_list or not has_new_quarters(quarter_list):
    print("No hay nueva información de quarters. Finalizando la ejecución.")
else:
    # Guardar los quarters actuales
    save_quarters(quarter_list)

    # Leer el DataFrame de filer_ids desde un archivo CSV
    filers = pd.read_csv('filers.csv')
    filers = filers.rename(columns={'id': 'filer_id'})

    # Lista para almacenar los holdings
    holdings_list = []

    # Iterar sobre todos los registros en filers
    for index, row in filers.iterrows():
        filer_id = row['filer_id']
        holdings = fetch_holdings(filer_id)
        holdings_list.extend(holdings)

    # Almacenar la información en un DataFrame
    holdings_df = pd.DataFrame(holdings_list)
    logging.info("Datos almacenados en el DataFrame")
    print(holdings_df)

# Calcular el tiempo de extracción
end_time = datetime.now()
extraction_time = (end_time - start_time).total_seconds()
logging.info(f"Tiempo total de extracción: {extraction_time} segundos")

In [ ]:
import pandas as pd
import mysql.connector

# Cargar CSV en DataFrame
df = pd.read_csv('quarters_DF.csv',sep=';')

# Conectar a MySQL
conn = mysql.connector.connect(
            host='localhost',
            database='Portafolio',
            user='root',
            password='P4ssw0rd!'
        )

cursor = conn.cursor()

# Strip leading/trailing spaces from the 'filing_period' column
df['filing_period'] = df['filing_period'].str.strip()

# Convert 'filing_period' to the format 'YYYY-MM-DD'
df['filing_period'] = pd.to_datetime(df['filing_period'], format='%m-%d-%Y').dt.strftime('%Y-%m-%d')

# Insert rows one by one
for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO quarters (quarter_id,filing_period,Status)
        VALUES (%s,%s,%s)
    """, (row['quarter_id'], row['filing_period'], row['Status']))

conn.commit()
conn.close()

In [ ]:
import time
import pandas as pd
import mysql.connector

# Cargar CSV en DataFrame
df = pd.read_csv('holding_df.csv', sep=',')

# Conectar a MySQL
conn = mysql.connector.connect(
            host='localhost',
            database='Portafolio',
            user='root',
            password='P4ssw0rd!'
        )

cursor = conn.cursor()

# Replace NaN values with None (interpreted as NULL in MySQL)
df = df.where(pd.notnull(df), None)

for _, row in df.iterrows():
    print("Valores esperados:", (
        row['filer_id'], row['filer_name'], row['stock_id'], row['stock_name'], row['stock_ticker'],
        row['security_type'], row['shares_change'], row['position_change_type'], 
        row['current_ranking'] if pd.notnull(row['current_ranking']) else None,
        row['previous_ranking'] if pd.notnull(row['previous_ranking']) else None,
        row['current_percent_of_portfolio'] if pd.notnull(row['current_percent_of_portfolio']) else None,
        row['previous_percent_of_portfolio'] if pd.notnull(row['previous_percent_of_portfolio']) else None,
        row['current_mv'] if pd.notnull(row['current_mv']) else None,
        row['previous_mv'] if pd.notnull(row['previous_mv']) else None,
        row['current_shares'] if pd.notnull(row['current_shares']) else None,
        row['previous_shares'] if pd.notnull(row['previous_shares']) else None,
        row['source_date'], row['source'], row['sector'], row['industry'], 
        row['percent_ownership'] if pd.notnull(row['percent_ownership']) else None,
        row['filer_street_address'], row['filer_city'], row['filer_state'], row['filer_zip_code'], 
        row['avg_price'] if pd.notnull(row['avg_price']) else None,
        row['percent_change'] if pd.notnull(row['percent_change']) else None,
        row['quarter_id_owned'] if pd.notnull(row['quarter_id_owned']) else None,
        row['quarter_end_price'] if pd.notnull(row['quarter_end_price']) else None
    ))
# # Insert rows one by one
# for _, row in df.iterrows():
#     cursor.execute("""
#         INSERT IGNORE INTO holding (filer_id, filer_name, stock_id, stock_name, stock_ticker, security_type, shares_change, position_change_type, current_ranking, previous_ranking, current_percent_of_portfolio, previous_percent_of_portfolio, current_mv, previous_mv, current_shares, previous_shares, source_date, source, sector, industry, percent_ownership, filer_street_address, filer_city, filer_state, filer_zip_code, avg_price, percent_change, quarter_id_owned, quarter_end_price)
#         VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
#     """, (
#         row['filer_id'], row['filer_name'], row['stock_id'], row['stock_name'], row['stock_ticker'],
#         row['security_type'], row['shares_change'], row['position_change_type'], 
#         row['current_ranking'] if pd.notnull(row['current_ranking']) else None,
#         row['previous_ranking'] if pd.notnull(row['previous_ranking']) else None,
#         row['current_percent_of_portfolio'] if pd.notnull(row['current_percent_of_portfolio']) else None,
#         row['previous_percent_of_portfolio'] if pd.notnull(row['previous_percent_of_portfolio']) else None,
#         row['current_mv'] if pd.notnull(row['current_mv']) else None,
#         row['previous_mv'] if pd.notnull(row['previous_mv']) else None,
#         row['current_shares'] if pd.notnull(row['current_shares']) else None,
#         row['previous_shares'] if pd.notnull(row['previous_shares']) else None,
#         row['source_date'], row['source'], row['sector'], row['industry'], 
#         row['percent_ownership'] if pd.notnull(row['percent_ownership']) else None,
#         row['filer_street_address'], row['filer_city'], row['filer_state'], row['filer_zip_code'], 
#         row['avg_price'] if pd.notnull(row['avg_price']) else None,
#         row['percent_change'] if pd.notnull(row['percent_change']) else None,
#         row['quarter_id_owned'] if pd.notnull(row['quarter_id_owned']) else None,
#         row['quarter_end_price'] if pd.notnull(row['quarter_end_price']) else None
#     ))
#     conn.commit()  # Commit after each insertion
#     print(f"Fila insertada: {row['filer_id']} - {row['stock_name']}")

conn.close()


In [ ]:
import time
import pandas as pd
import mysql.connector

# Cargar CSV en DataFrame
df = pd.read_csv('holding_df.csv', sep=',')

# Conectar a MySQL
conn = mysql.connector.connect(
            host='localhost',
            database='Portafolio',
            user='root',
            password='P4ssw0rd!'
        )

cursor = conn.cursor()

# Replace NaN values with None (interpreted as NULL in MySQL)
df = df.where(pd.notnull(df), None)

# Imprimir las columnas del DataFrame
print("Columnas del DataFrame:", df.columns)

# Verificar valores esperados y columnas faltantes
for _, row in df.iterrows():
    try:
        valores = (
            row['filer_id'], row['filer_name'], row['stock_id'], row['stock_name'], row['stock_ticker'],
            row['security_type'], row['shares_change'], row['position_change_type'], 
            row['current_ranking'] if pd.notnull(row['current_ranking']) else None,
            row['previous_ranking'] if pd.notnull(row['previous_ranking']) else None,
            row['current_percent_of_portfolio'] if pd.notnull(row['current_percent_of_portfolio']) else None,
            row['previous_percent_of_portfolio'] if pd.notnull(row['previous_percent_of_portfolio']) else None,
            row['current_mv'] if pd.notnull(row['current_mv']) else None,
            row['previous_mv'] if pd.notnull(row['previous_mv']) else None,
            row['current_shares'] if pd.notnull(row['current_shares']) else None,
            row['previous_shares'] if pd.notnull(row['previous_shares']) else None,
            row['source_date'], row['source'], row['sector'], row['industry'], 
            row['percent_ownership'] if pd.notnull(row['percent_ownership']) else None,
            row['filer_street_address'], row['filer_city'], row['filer_state'], row['filer_zip_code'], 
            row['avg_price'] if pd.notnull(row['avg_price']) else None,
            row['percent_change'] if pd.notnull(row['percent_change']) else None,
            row['quarter_id_owned'] if pd.notnull(row['quarter_id_owned']) else None,
            row['quarter_end_price'] if pd.notnull(row['quarter_end_price']) else None
        )
        print("Valores esperados:", valores)
    except KeyError as e:
        print(f"Error: Falta la columna {e} en el DataFrame.")
        break

# # Insert rows one by one
# for _, row in df.iterrows():
#     cursor.execute("""
#         INSERT IGNORE INTO holding (filer_id, filer_name, stock_id, stock_name, stock_ticker, security_type, shares_change, position_change_type, current_ranking, previous_ranking, current_percent_of_portfolio, previous_percent_of_portfolio, current_mv, previous_mv, current_shares, previous_shares, source_date, source, sector, industry, percent_ownership, filer_street_address, filer_city, filer_state, filer_zip_code, avg_price, percent_change, quarter_id_owned, quarter_end_price)
#         VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
#     """, valores)
#     conn.commit()  # Commit after each insertion
#     print(f"Fila insertada: {row['filer_id']} - {row['stock_name']}")

conn.close()

In [ ]:
import pandas as pd
import mysql.connector

# Cargar CSV en DataFrame
# Use on_bad_lines='skip' to skip problematic rows
df = pd.read_csv('filers.csv')
# Alternatively, if you want to debug the issue, you can read the file without parsing and inspect it:
# with open('filers.csv', 'r') as file:
#     lines = file.readlines()
#     for i, line in enumerate(lines):
#         print(f"Line {i + 1}: {line.strip()}")

# Conectar a MySQL
conn = mysql.connector.connect(
            host='localhost',
            database='Portafolio',
            user='root',
            password='P4ssw0rd!'
        )

cursor = conn.cursor()

# Replace NaN values with None (interpreted as NULL in MySQL)
df = df.where(pd.notnull(df), None)

# Insert rows one by one and commit after each insertion
for _, row in df.iterrows():
    cursor.execute("""
        INSERT IGNORE INTO filers (filer_id,name,cik,street_address,state,zip_code,business_phone,permalink)
        VALUES (%s,%s,%s,%s,%s,%s,%s,%s)
    """, (row['id'], row['name'], row['cik'], row['street_address'], row['state'], row['zip_code'], row['business_phone'], row['permalink']))
    conn.commit()  # Commit after each insertion

conn.commit()
conn.close()


In [ ]:
import mysql.connector

try:
    conn = mysql.connector.connect(
            host='localhost',
            database='Portafolio',
            user='root',
            password='P4ssw0rd!'
        )
    cursor = conn.cursor()

    # Ejecutar alguna consulta
    cursor.execute("""
ALTER TABLE `Portafolio`.`holding` 
CHANGE COLUMN `previous_share` `previous_shares` INT NULL DEFAULT NULL ;
                   """)


except mysql.connector.Error as err:
    print("Error:", err)

finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()  #  Esta línea cierra la conexión

In [ ]:
x =df[df['source_date']=='2025-04-30']
x